In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import requests
from typing import List

app = FastAPI()

# Database simulation
users = {}
inventory = []

# Models
class User(BaseModel):
    name: str
    age: int
    weight: float
    height: float
    activity_level: str
    dietary_preferences: List[str] = []
    allergies: List[str] = []

class FoodItem(BaseModel):
    barcode: str
    name: str
    quantity: str
    categories: str
    expiration_date: str

# 1️⃣ User Data Management
@app.post("/user/create")
def create_user(user: User):
    users[user.name] = user
    return {"message": "User created successfully!", "user": user}

@app.get("/user/{name}")
def get_user(name: str):
    user = users.get(name)
    if not user:
        raise HTTPException(status_code=404, detail="User not found")
    return user

# 2️⃣ Diet Calculator (Basic Example)
@app.get("/diet/calculate")
def calculate_diet(age: int, weight: float, height: float, activity_level: str):
    bmr = 10 * weight + 6.25 * height - 5 * age + 5  # Mifflin-St Jeor Equation
    activity_multiplier = {"sedentary": 1.2, "active": 1.55, "very active": 1.9}
    calories = bmr * activity_multiplier.get(activity_level, 1.2)
    macros = {"calories": calories, "protein": weight * 1.6, "carbs": calories * 0.5 / 4, "fats": calories * 0.3 / 9}
    return macros

# 3️⃣ Inventory Management
@app.post("/inventory/add")
def add_inventory(item: FoodItem):
    inventory.append(item)
    return {"message": "Item added to inventory", "item": item}

@app.get("/inventory")
def get_inventory():
    return inventory

# 4️⃣ Barcode Lookup
@app.get("/barcode/{barcode}")
def get_product_info(barcode: str):
    url = f"https://world.openfoodfacts.org/api/v2/product/{barcode}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "product" in data:
            product_name = data["product"].get("product_name", "N/A")
            categories = data["product"].get("categories", "N/A")
            return {"product_name": product_name, "categories": categories}
    raise HTTPException(status_code=404, detail="Product not found")
